# Copying figures from Holst et al "Impacts of summer water limitation on the carbon balance of a Scots pine forest in the southern upper Rhine plain"

In [ ]:
# Settings
Username   = 'Beheerder'
years      = range(2001,2020)    #(1997,2021) # Set years to download

In [ ]:
import os
datapath   = os.path.join('../')
print('datapath is set to %s'%datapath)

# !pip install numpy
# !pip install pandas
# !pip install matplotlib
# !pip install plotly 
# !pip install cufflinks
#!pip install colorspacious
#!pip install seaborn

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
#import plotly.express as px
#import cufflinks as cf
import matplotlib.dates as mdate
import matplotlib.ticker as ticker
from matplotlib import cm
#from colorspacious import cspace_converter
import scipy.stats as stats
#cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

from datetime import datetime, timedelta
import sys
sys.path.insert(0, os.path.join(datapath,'PythonScripts'))
from Loobos_Toolbox import dateparse, dateparse_Gapfilled, Read_LoobosEddFinal, Read_LooStor, Read_LoodatGapfill, Read_Loobos_halfhourly, Read_Loobos_meteo, Read_Loobos_soil, Read_Loobos_profile

from Ags_model import runAgs, calc_LE

In [ ]:
from FilterData import Filter_wrap

In [ ]:
#these next two lines are to prevent re-loading the data. If you want to re-load data, instead comment them out
if not 'progress' in globals(): progress = list()
if not 'dataloaded' in progress:
  # Read files
    df_EC           = Read_LoobosEddFinal    (years,datapath)
    df_Stor         = Read_LooStor           (years,datapath)
    df_Comb         = Read_LoodatGapfill     (years,datapath)
    df_NEE          = Read_Loobos_halfhourly (years,datapath)
    df_meteo        = Read_Loobos_meteo      (years,datapath)
    df_soil         = Read_Loobos_soil       (years,datapath) 
    df_profile      = Read_Loobos_profile    (years,datapath)
    progress.append('dataloaded')

In [ ]:
#from FilterData import Filter_wrap
CO2,Locorr,VPD,Ustar,df_profile_filter,df_meteo_filter,df_Comb_filter,df_EC_filter=Filter_wrap(df_Comb,df_profile,df_meteo,df_EC,filterversion='default')
#NOTE: df_Stor is NOT FILTERED

# Start of figure generation

## Figure 2

In [ ]:
#set aspect ratio and dpi
aspectratio=21/9
fig_wid=6 #we want 6 inch width
print(fig_wid/aspectratio) # around 2.6
fig_hei=2.6
dpi=144 #can be as high as 300, standard is 72 which is way too low

In [ ]:
# plotting F_NEE(micromol /m2 /s) over Ta (deg C), Ts (deg C), VPD (hPa), separated into two classes: PAR>1000 and PAR500-1000
#reminder to self: maybe make a LUE curve to justify the classes >1000 and 500-1000? Or unnecessary, quote literature?

#F_NEE data sourced from: df_Comb: NEE
#VPD data sourced from: df_Comb: VPD
#Ta (air temp) data sourced from: df_Comb: Tair
#Ts (soil temp) data sourced from: df_Comb: Tsoil
#PAR data sourced from: df_meteo: PAR

df_fig2 = pd.concat([df_Comb_filter['NEE'],df_Comb_filter['VPD'],df_Comb_filter['Tair'],df_Comb_filter['Tsoil'], df_meteo_filter['PAR']],axis=1,sort=False)

In [ ]:
df_fig2.plot()

In [ ]:
#df_test = df_fig2.loc[df_fig2['PAR']<1000]
#df_test2 = df_test.loc[df_test['PAR']>500]

#df_test2e = df_fig2.loc[(500<df_fig2['PAR']<1000)]

#df_test2e.equals(df_test2)
#this throws error

In [ ]:
#fig 2 e)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2e = df_fig2.loc[df_fig2['PAR']<1000]

fig,ax = plt.subplots()
#p1=ax.scatter(df_tmp1['GPP_f'],df_tmp1['sim_data_1'],c=df_tmp1.index.hour,cmap='viridis') #scatter(x,y)
p1=ax.scatter(df_fig2e['VPD'],df_fig2e['NEE'],marker='o',c=df_fig2e['Tair'],cmap='viridis') #scatter(x,y)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference')
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')
ax.set_ylim(-45,30)
ax.set_xlim(-5,45)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'VPD [Pa or hPA?]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2e: NEE vs VPD (color by Ta), PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  
cbar=plt.colorbar(p1,ax=ax)
cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(144)
plt.plot()

In [ ]:
#fig 2 f)

#slope1, intercept1, r_value1, p_value1, std_err1 = stats.linregress(df_tmp1['sim_data_1'], df_tmp1['sim_data_2'])  #linregres x, y . note r_value is Pearson's coefficient. R^2 is r_value**2
#print('slope, intercept, R2:', slope1, intercept1, r_value1**2)

df_fig2f = df_fig2.loc[df_fig2['PAR']<1000]
df_fig2f = df_fig2f.loc[df_fig2f['PAR']>500]
#make aliases
Tair=df_fig2f['Tair']
NEE=df_fig2f['NEE']
VPD=df_fig2f['VPD']

#because the marker argument in scatter() doesn't accept a long list the same size of the data where each entry specifies the marker of that point (unlike c - color arg),
#we have to split the dataset into the categories and plot it over each other with multiple instances of scatter()

#separate data:
#important! check if lengths of each of these are the same
fig2f_cat1=df_fig2f.loc[df_fig2f['Tair']<=15]
fig2f_cat2=df_fig2f.loc[df_fig2f['Tair']>15]

#colorbar settings
cmap = mpl.colormaps['coolwarm'].resampled(2)
cols = [mpl.colors.to_hex(cmap(0)),mpl.colors.to_hex(cmap(1))]

cbar_bounds = [10, 15 ,20] #boundaries excluding the abs min and abs max
#cbar_bounds = [10, 15, 20, 30] #old values
norm = mpl.colors.BoundaryNorm(cbar_bounds, cmap.N)

fig,ax = plt.subplots()
ax_cbar = fig.add_axes([0.05, -0.20, 0.9, 0.05])
p1=ax.scatter(fig2f_cat1['VPD'],fig2f_cat1['NEE'],marker='o',c=cols[0],alpha=0.5) #scatter(x,y)
p2=ax.scatter(fig2f_cat2['VPD'],fig2f_cat2['NEE'],marker='d',c=cols[1],alpha=0.5) #scatter(x,y)

#trendlines (work in progress)
#ax.axline((0.0,intercept1),slope=slope1,c='black',label='reference') 
#ax.axline ((0.0,0.0), slope=1, c='r',linestyle='dashed',label='1:1')

ax.set_ylim(-45,30)
ax.set_xlim(-1.5,45)
ax.set_ylabel(r'NEE [$\mu molm^{-2}s^{-1}$] (?)')
ax.set_xlabel(r'VPD [Pa or hPA?]')
#fig.suptitle('Correlation of increased VPD to reference run for An, \n (2008-2017 during growth seasons May-Oct.)')
ax.set_title('Fig2f: NEE vs VPD (color by Ta), 500<PAR<1000')
#ax.legend(loc='upper left')
#fig.subplots_adjust(top=0.87) #fix the top margin text overlap  

cb2 = mpl.colorbar.ColorbarBase(ax_cbar, cmap=cmap,
                                norm=norm,
                                # to use 'extend', you must specify two extra boundaries:
                                boundaries=[10]+cbar_bounds+[20],
                                extend='both',
                                ticks=cbar_bounds,  # optional
                                spacing='proportional',
                                orientation='horizontal')
cb2.set_label('Tair')
#cbar=plt.colorbar(p1,ax=ax)
#cbar.ax.set_ylabel('Tair')
fig.set_figwidth(6)
fig.set_figheight(2.6)
fig.set_dpi(144)
plt.plot()

In [ ]:
#adapted figure: plot GPP instead of NEE (thus eliminating effects respiration, as NEE = GPP - respiration (ecosys.)

#GPP data sourced from: df_Comb: GPP_f

### end of figure 2

## figure 3

### end of figure 3